# **UCalgary ASI Exercises**

First we're going to do the pip installs and imports for you so you can get rolling a bit quicker. Scroll down to see the individual exercise descriptions.

In [4]:
!pip install themis-imager-readfile rego-imager-readfile trex-imager-readfile aacgmv2 cartopy pyproj matplotlib scipy requests joblib tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 69.7 MB/s eta 0:00:00
  Created wheel for aacgmv2: filename=aacgmv2-2.6.3-cp310-cp310-linux_x86_64.whl size=1679869 sha256=6391e577112c713e66b041afdc753006be5d508de1cb3397373ff6574ca8f993
  Stored in directory: /root/.cache/pip/wheels/e9/d6/3f/10a359ebb903b2dbe5aa94f7024b5284059bb0daef65a7a0b4
Successfully built aacgmv2


In [5]:
import os
import pprint
import glob
import datetime as dt
import themis_imager_readfile
import rego_imager_readfile
import trex_imager_readfile
import aacgmv2
import cv2
import cartopy
import pyproj
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import scipy
import requests
import joblib
from tqdm.notebook import tqdm

## **Exercise 1: Growth rate of an instability  [Easy]**

One of the first substorms with an onset captured by THEMIS ASI, and for which the satellites were aligned to track the evolution of the event near the equatorial plane, occurred on March 13, 2007, over The Pas, a bit after 05 UT. This event was explored in the Donovan et al. 2008 GRL (https://agupubs.onlinelibrary.wiley.com/doi/10.1029/2008GL033794).

This paper highlighted the period of linear growth of brightness of the onset arc that we associate with the growth of the onset instability in the tail.

Using only THEMIS-ASI data from The Pas, create a plot similar to the bottom panel of Figure 4 in Donovan's paper. The key point is to isolate the right geomagnetic latitude range and the right meridians (to do this well). However, one can actually just use the images without mapping, if one is careful, and use one meridian to do what we are suggesting, namely to estimate the growth rate (or growth time scale).

In [6]:
# set the top-level output path we want to save the files to
#
# NOTE: This crib sheet assumes you're using Google Colab. Change as needed.
download_root_path = "/content/ucalgary_data"

In [7]:
# These functions are helpers for downloading data, we won't go into any detail
# here, but just use them later on. Learn more by checking out the 'Downloading data
# from the API' crib sheet on data.phys.ucalgary.ca.

def download_url(url, prefix, output_base_path, overwrite=False, pbar=None):
    # set output filename
    output_filename = "%s/%s" % (output_base_path, url.removeprefix(prefix))
    if (overwrite is False and os.path.exists(output_filename)):
        if (pbar is not None):
            pbar.update()
        return

    # create destination directory
    try:
        os.makedirs(os.path.dirname(output_filename), exist_ok=True)
    except Exception:
        # NOTE: sometimes when making directories in parallel there are race conditions. We put
        # in a catch here and carry on if there are ever issues.
        pass

    # retrieve file and save to disk
    r = requests.get(url)
    with open(output_filename, 'wb') as fp:
        fp.write(r.content)

    # advance progress bar
    if (pbar is not None):
        pbar.update()

def download_urls(dataset, urls, output_base_path, n_parallel=5, overwrite=False):
    prefix_to_strip = dataset["data_tree_url"]
    with tqdm(total=len(urls), desc="Downloading and saving files to disk") as pbar:
        joblib.Parallel(n_jobs=n_parallel, prefer="threads")(
            joblib.delayed(download_url)(
                urls[i],
                prefix_to_strip,
                os.path.join(output_base_path, dataset["name"]),
                overwrite=overwrite,
                pbar=pbar,
            ) for i in range(0, len(urls))
        )
    print("\nData saved to %s" % (output_base_path))

def get_data_urls(dataset_name, start, end, site_uid):
    params = {"name": dataset_name, "start": start, "end": end, "site_uid": site_uid, "include_total_bytes": True}
    r = requests.get("https://api.phys.ucalgary.ca/api/v1/data_distribution/urls", params=params)
    data = r.json()
    return data

def list_datasets(name=None):
    if (name is not None):
        params = {"name": name}
        r = requests.get("https://api.phys.ucalgary.ca/api/v1/data_distribution/datasets", params=params)
    else:
        r = requests.get("https://api.phys.ucalgary.ca/api/v1/data_distribution/datasets")
    datasets = r.json()
    return datasets

In [9]:
# download a couple hours of data
dataset_name = "THEMIS_ASI_RAW"
start = "2007-03-13T04:00"
end = "2007-03-13T05:59"
site_uid = "tpas"
dataset = list_datasets(name=dataset_name)[0]
urls = get_data_urls(dataset_name, start, end, site_uid)["urls"]
download_urls(dataset, urls, download_root_path)


Data saved to /content/ucalgary_data


### **Start your work here**

In [10]:
# put code here


---
## **Exercise #2: Creating a 557.7nm mosaic from TREx-RGB [Easy]**

The RGB data is broadband color, but recent work by Jun Liang has shown that using the Green channel of RGB, along with a calibration curve, you can estimate the absolute luminosity of 557.7nm.  This is useful for making maps of color ratios (557.7nm and 630.nm, related to electron energy flux).

Using the information in Liang et al. 2023 (https://www.eppcgs.org/en/article/doi/10.26464/epp2023063) construct a calibrated 557.7nm mosaic for a single timeframe.

Hint: in this case, you need to turn a multi-channel imager into a single channel imager. So the single channel mosaic crib sheet is relevant, but you need to load the RGB data and apply the procedure documented in the paper.


In [11]:
# put code here


---
## **Exercise #3: M-number of 'the beads'  [Easy]**

The so-called "auroral beads" were first identified Eric Donovan's ICS-8 Proceedings paper: https://ics8.ca/proc_files/donovan.pdf. In Figure 5 of that paper, you can see a "difference plot," where the color indicates the difference in brightness between the current and previous image. The data comes from Fort Yukon, and spans 101148 to 101254 UT on November 28, 2005.

Make a difference plot, similar to that in Donovan's paper, highlighting the evolution of the beads. Assuming the elevation is linearly increasing with distance from the edge of the image (in pixel coordinates), and the aurora is at 110 km altitude, estimate the m-number of the instability.

Note: you can use the API, or alternatively, you can go to our data tree and download the two relevant files directly.
(https://data.phys.ucalgary.ca/sort_by_project/THEMIS/asi/stream0/2005/11/28/fykn_themis01/ut10/)

In [12]:
# put code here


## **Exercise #4: Making a multi-imager keogram  [Challenging]**

Keograms are extremely useful for understanding magnetic topology. For example, the polar cap boundary (or polar ward edge) of the redline aurora is often used to mark the open-closed field line boundary. Similarly, the equatorward edge of the auroral oval can be thought of as the inner boundary of plasma sheet scattering processes. When looking at the whole system, from polar cap to sub-auroral, you often need more than one imager.

In this exercise, we challenge you to make a combined, continuous keogram from two REGO imagers separated in latitude (Gillam and Rankin Inlet). This will truly visualize the ionospheric projection of the plasma sheet.
We have included a few good hours of data to do this with in the google drive.  Look for 2023-02-20 from REGO (and don't forget to double check using the data portal, data.phys.ucalgary.ca).

Methodology:  You will want to work in geomagnetic latitude, create a latitude and time 2D array. Your array should span the latitude range of Rankin Inlet and Gillam REGO FoVs (in geomagnetic!).  Then carefully fill your grid with the appropriate values from the image data.

Things to watch out for:  Scaling. You can do this in data numbers, or calibrated luminosity. If in data numbers, you will want to subtract the dark and possibly independently scale the imagers (due to their differing local light conditions).  Also watch out for how you choose which imager to use at what latitude.

In [13]:
# put code here